In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Data source에서 d3 와 d4 데이터를 활용하기로 결정
### d3 데이터에서 train 데이터와 test 데이터를 합치고, d4 데이터와 겹치는 columns을 이용하기 위한 전처리 진행
### d4 데이터 출처 github에서 완전한 rawdata 를 가져오고, d3 데이터와 겹치는 columns를 이용하기 위한 전처리 진행

In [ ]:
d3_train = pd.read_csv("../input/term-project-final-rawdata/d3_train-data.csv")
d3_test = pd.read_csv("../input/term-project-final-rawdata/d3_test-data.csv")
companies = pd.read_csv("../input/term-project-final-rawdata/companies.csv")

### 1. d3 데이터 전처리

In [ ]:
# d3 데이터 합치기
d3_data = pd.concat([d3_train, d3_test], axis = 0)

In [ ]:
# d3 데이터 전처리
d3_data = d3_data.drop(['Unnamed: 0','state_code', 'closed_at','latitude', 'longitude', 'milestones','zip_code', 'id', 'Unnamed: 6',
                        'labels','age_first_funding_year','age_last_funding_year','age_first_milestone_year','age_last_milestone_year',
                       'object_id', 'has_VC', 'has_angel', 'has_roundA','has_roundB', 'has_roundC', 'has_roundD', 'avg_participants',
                        'relationships','is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate','is_software', 'is_web', 'is_mobile',
       'is_enterprise', 'is_advertising', 'is_gamesvideo', 'is_ecommerce',
       'is_biotech', 'is_consulting', 'is_othercategory',
       'is_top500'], axis = 1)

In [ ]:
# d3_data에 'country_code' column 추가 (미국 데이터 이므로 국가코드는 USA로)
# d3_data = d3_data.insert(1, 'country_code', 'USA')

d3_data['country_code'] = 'USA'

# DataFrame columns 들 순서 변경
d3_data = d3_data[['name', 'country_code',  'state_code.1', 'city', 'category_code', 'founded_at', 
                   'first_funding_at', 'last_funding_at', 'funding_rounds', 'funding_total_usd', 'status']]

# d3, d4 column명 통일시키기
d3_data = d3_data.rename(columns = {"state_code.1" : "state_code", "category_code" : "market"})
d3_data.head()

d3_data.head()

In [ ]:
d3_data.info()

### 2. d4 데이터 전처리

In [ ]:
# d3 데이터와 column 이 맞도록 전처리
companies = companies.drop(['permalink', 'homepage_url', 'region'], axis = 1)

In [ ]:
# category_list 에서 카테고리를 하나만 갖도록 수정, 편의상 첫번째로 오는 카테고리를 갖도록 처리
def one_category(tmp):
    
    def is_null(tmp2):
        if tmp2 != tmp2:
            return True
    
    if is_null(tmp) == True:
        return tmp
    else:
        new = tmp.split('|')
        return new[0]

companies.category_list = companies.category_list.apply(one_category)

# d3, d4 column명 통일시키기
companies = companies.rename(columns = {"category_list" : "market"})

d4_data = companies[['name', 'country_code', 'state_code', 'city', 'market', 'founded_at', 
                   'first_funding_at', 'last_funding_at', 'funding_rounds', 'funding_total_usd', 'status']]

d4_data.head()

In [ ]:
d4_data.info()

In [ ]:
print(d4_data['founded_at'])

In [ ]:
d4_data['founded_at'].isnull().sum()

In [ ]:
d4_data = d4_data[d4_data['funding_total_usd'] != '-']
d4_data = d4_data[d4_data['founded_at'] != 'NaN']
d4_data = d4_data.dropna(subset = ['founded_at'])


In [ ]:
print(d4_data.shape)

In [ ]:
d4_data.info()

### 3. d3 + d4 data

In [ ]:
# 데이터는 d3 와 d4 합친것을 사용
data = pd.concat([d3_data, d4_data], axis = 0, ignore_index = True)
data.head()

In [ ]:
# version1 데이터상의 오류 발견 => d3 d4를 concat하고 shuffle 하지 않음
# version2: d3, d4 데이터 shuffle
# version3: d4 데이터 완전한 raw 데이터로 새로 만듬

# sklearn.utils.shuffle 을 이용해 shuffle, random_state 77로 고정
from sklearn.utils import shuffle
data = shuffle(data, random_state = 77)

In [ ]:
data.head()

In [ ]:
data['id'] = data.index
data = data[data['funding_total_usd'] != '-']
data.info()

In [ ]:
data = data[['id', 'name', 'country_code',  'state_code', 'city', 'market', 'founded_at', 
                   'first_funding_at', 'last_funding_at', 'funding_rounds', 'funding_total_usd', 'status']]

In [ ]:
data.head()

In [ ]:
print(data['status'].unique())

## train과 test 데이터 제작

In [ ]:
data.shape

In [ ]:
test_size = int(47000*(0.2))
test_size

In [ ]:
data = data.iloc[:47001,:]
train = data.iloc[:47000-test_size,:]
test = data.iloc[47000-test_size:-1, :]
print(train.shape)
print(test.shape)

In [ ]:
test.head()

## 캐글 리더보드에 설정할 solution data 제작

In [ ]:
solution = pd.DataFrame(test['status'])
solution['id'] = test.index
solution = solution[['id', 'status']]
solution.head()

In [ ]:
# solution 파일에 status가 'closed' 일때 0 값이 들어가도록 매핑

print(solution['status'].unique())

def return_Category(tmp):
    if tmp == 'closed':
        return 0
    else:
        return 1

solution['status'] = solution['status'].apply(return_Category)
solution.head()

## Submit 파일 제작

In [ ]:
# make submission file

index = test.index
fill_tmp_status = 777

submit = pd.DataFrame({'index' : index, 'status': fill_tmp_status})
submit.head()

## 제작한 모든 파일 출력

In [ ]:
test = test.drop('status', axis = 1)

train.to_csv("startup_train.csv", index = False)
test.to_csv("startup_test.csv", index = False)
solution.to_csv("17011805_solution.csv", index = False)
submit.to_csv("submit.csv", index = False)

In [ ]:
print('DATA MAKING COMPLETE')